In [10]:
from jenkspy import JenksNaturalBreaks
import numpy as np
import pandas as pd

In [11]:
def govf(jnb, arr):
    sdam = np.sum((arr - arr.mean()) ** 2)
    sdcm = sum([np.sum((g - g.mean()) ** 2) for g in jnb.groups_])
    return (sdam - sdcm) / sdam

In [12]:
def binit(ser: pd.Series, MAXCAT: int) -> (pd.Series, object):
    opt = MAXCAT
    for k in range(2,MAXCAT):
        jnb = JenksNaturalBreaks(k)
        arr = np.array(ser)
        jnb.fit(arr)
        thresh = govf(jnb,arr)
        if(thresh>0.8):
            opt = k
            break

    jnb = JenksNaturalBreaks(opt)
    arr = np.array(ser)
    jnb.fit(arr)
    lab = jnb.labels_
    breaks = ["-inf"]+ [str(i) for i in jnb.inner_breaks_]  + ["+inf"]
    ans = []
    for i in lab:
        ans = ans + [breaks[i] + "< "+ ser.name +" <"+ breaks[i+1]]
    return (pd.Series(ans),jnb)

In [16]:
def process(data, name):
    var = data[name]
    var_notna = var[var.notna()]
    var_, jnb_ = binit(var_notna,10)
    var_cleaned = []
    counter = 0
    for i in range(len(var)):
        if(pd.notna(var[i])):
            var_cleaned += [var_[i-counter]]
        else:
            var_cleaned += ["None"]
            counter = counter+1
    return pd.Series(var_cleaned,name=name)

In [25]:
from sklearn.datasets import load_iris
data = load_iris(as_frame=True).data

In [27]:
for i in data.columns:
    data[i] = process(data,i)

In [102]:
new_dat = pd.DataFrame(columns={"Items"})
new_dat["Items"] = data.apply(lambda x: tuple(dict(x).items()), axis=1)
new_dat.index.rename("id")
new_dat = new_dat.reset_index()

In [103]:
new_dat

,index,Items
0,0,"[(sepal length (cm), -inf< sepal length (cm) <..."
1,1,"[(sepal length (cm), -inf< sepal length (cm) <..."
2,2,"[(sepal length (cm), -inf< sepal length (cm) <..."
3,3,"[(sepal length (cm), -inf< sepal length (cm) <..."
4,4,"[(sepal length (cm), -inf< sepal length (cm) <..."
...,...,...
145,145,"[(sepal length (cm), 6.4< sepal length (cm) <+..."
146,146,"[(sepal length (cm), 5.4< sepal length (cm) <6..."
147,147,"[(sepal length (cm), 6.4< sepal length (cm) <+..."
148,148,"[(sepal length (cm), 5.4< sepal length (cm) <6..."


new_dat.Items

In [104]:
from pycaret.arules import *
arules = setup(new_dat, transaction_id = 'index', item_id = 'Items')

TypeError: unhashable type: 'list'

In [97]:
new_dat["Items"][0]

(('sepal length (cm)', '-inf< sepal length (cm) <5.4'),
 ('sepal width (cm)', '3.3< sepal width (cm) <+inf'),
 ('petal length (cm)', '-inf< petal length (cm) <3.0'),
 ('petal width (cm)', '-inf< petal width (cm) <0.6'))

In [101]:

create_model(metric = 'lift', threshold = 0.000001)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
